In [10]:
# Install necessary packages
!pip install pandas numpy arch statsmodels openpyxl

import pandas as pd
import numpy as np
from arch import arch_model
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [11]:
# Load the dataset
data = pd.read_csv("D:\\SCMA\\Data\\AMZN.csv")

In [12]:
# Convert the Date column to Date type
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

In [13]:
# Extract the adjusted closing prices
adj_close = data['Adj Close']

In [14]:
# Calculate the log returns
log_returns = np.diff(np.log(adj_close))

In [15]:
# Perform ARCH test
arch_test = sm.stats.diagnostic.acorr_ljungbox(log_returns, lags=[1], return_df=True)
print(arch_test)


    lb_stat  lb_pvalue
1  0.091093   0.762792


In [16]:
# Fit an appropriate GARCH model
# Here we choose a GARCH(1,1) model
model = arch_model(log_returns, vol='Garch', p=1, q=1)
garch_fit = model.fit()
print(garch_fit.summary())

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2869439.7112477226
Iteration:      2,   Func. Count:     17,   Neg. LLF: 161663.34958870144
Iteration:      3,   Func. Count:     28,   Neg. LLF: 68.457512559215
Iteration:      4,   Func. Count:     37,   Neg. LLF: 20.306666549242813
Iteration:      5,   Func. Count:     46,   Neg. LLF: -593.5908045323522
Iteration:      6,   Func. Count:     52,   Neg. LLF: -653.9728337365339
Iteration:      7,   Func. Count:     59,   Neg. LLF: -643.2393935962291
Iteration:      8,   Func. Count:     65,   Neg. LLF: -663.750568817277
Iteration:      9,   Func. Count:     71,   Neg. LLF: -663.7771669270132
Iteration:     10,   Func. Count:     76,   Neg. LLF: -663.7773158745773
Iteration:     11,   Func. Count:     81,   Neg. LLF: -663.7773226440474
Iteration:     12,   Func. Count:     85,   Neg. LLF: -663.7773226440227
Optimization terminated successfully    (Exit mode 0)
            Current function value: -663.7773226440474
            Iterati

C:\Users\HP\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002992. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [17]:
# Forecast the three-month volatility
# Assuming 21 trading days per month, 3 months would be 63 trading days
forecast = garch_fit.forecast(horizon=63)

In [18]:
# Extract the forecasted volatility
forecasted_volatility = np.sqrt(forecast.variance.values[-1, :])
print(forecasted_volatility)

[0.01763557 0.01787217 0.01807787 0.01825702 0.01841327 0.01854971
 0.01866899 0.01877336 0.01886475 0.01894483 0.01901504 0.01907663
 0.01913067 0.01917812 0.01921979 0.01925639 0.01928856 0.01931682
 0.01934167 0.01936351 0.01938272 0.01939961 0.01941447 0.01942754
 0.01943903 0.01944914 0.01945804 0.01946587 0.01947276 0.01947882
 0.01948415 0.01948885 0.01949298 0.01949661 0.01949981 0.01950263
 0.01950511 0.01950729 0.01950921 0.0195109  0.01951238 0.01951369
 0.01951484 0.01951586 0.01951675 0.01951754 0.01951823 0.01951884
 0.01951937 0.01951984 0.01952026 0.01952062 0.01952095 0.01952123
 0.01952148 0.0195217  0.01952189 0.01952206 0.01952221 0.01952234
 0.01952246 0.01952256 0.01952265]


### Part B : VAR/VECM

In [19]:
# Import necessary libraries
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM

In [21]:
# Load the dataset
df = pd.read_excel('D:\\SCMA\\Data\\pinksheet.xlsx', sheet_name="Monthly Prices", skiprows=6)

In [22]:
# Display the first few rows of the dataset
print(df.head())

  Unnamed: 0  CRUDE_PETRO  CRUDE_BRENT  CRUDE_DUBAI CRUDE_WTI COAL_AUS  \
0    1960M01         1.63         1.63         1.63         …        …   
1    1960M02         1.63         1.63         1.63         …        …   
2    1960M03         1.63         1.63         1.63         …        …   
3    1960M04         1.63         1.63         1.63         …        …   
4    1960M05         1.63         1.63         1.63         …        …   

  COAL_SAFRICA  NGAS_US  NGAS_EUR NGAS_JP  ...    ALUMINUM  IRON_ORE  COPPER  \
0            …     0.14  0.404774       …  ...  511.471832     11.42  715.40   
1            …     0.14  0.404774       …  ...  511.471832     11.42  728.19   
2            …     0.14  0.404774       …  ...  511.471832     11.42  684.94   
3            …     0.14  0.404774       …  ...  511.471832     11.42  723.11   
4            …     0.14  0.404774       …  ...  511.471832     11.42  684.75   

    LEAD     Tin  NICKEL   Zinc   GOLD  PLATINUM  SILVER  
0  206.1  2180.

In [23]:
# Rename the first column to "Date"
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

In [24]:
# Check for missing values
print(df.isnull().sum())

Date           0
CRUDE_PETRO    0
CRUDE_BRENT    0
CRUDE_DUBAI    0
CRUDE_WTI      0
              ..
NICKEL         0
Zinc           0
GOLD           0
PLATINUM       0
SILVER         0
Length: 72, dtype: int64


In [25]:
# Fill or drop missing values as needed
# Here, we'll use forward fill method
df.fillna(method='ffill', inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_29104\1907699699.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [26]:
# Select relevant columns (example columns based on your task description)
# Update the column names as per your dataset
columns = ['CRUDE_PETRO', 'CRUDE_BRENT', 'CRUDE_DUBAI','SUGAR_EU', 'SUGAR_US','GOLD', 'PLATINUM', 'SILVER']


In [27]:
data = df[columns]

In [28]:
# Strip any leading/trailing spaces from column names
# df.columns = df.columns.str.strip()

In [29]:
# Display the cleaned column names
# print("DataFrame Columns:")
# print(df.columns)

In [30]:
# Update the list of columns to match the actual column names
# columns = ['Crude oil, average', 'Crude oil, Brent', 'Crude oil, Dubai', 'Crude oil, WTI', 
#            'Coal, Australian', 'Coal, South African **', 'Natural gas, US', 
#            'Natural gas, Europe', 'Liquefied natural gas, Japan', 'Natural gas index']

In [31]:
# Display the list of columns
# print("Expected Columns:")
# print(columns)

In [32]:
# Check if all columns are present in the DataFrame
# missing_columns = [col for col in columns if col not in df.columns]
# if missing_columns:
#     print(f"Missing columns: {missing_columns}")

#     # Investigate why the columns are missing
#     for col in columns:
#         if col not in df.columns:
#             print(f"Column '{col}' is missing from DataFrame columns.")
# else:
#     print("All columns are present in the DataFrame.")

#     # Select relevant columns
#     data = df[columns]

#     # Ensure all data is numeric
#     data = data.apply(pd.to_numeric)

#     # Display the first few rows of the selected data
#     print(data.head())

In [33]:
# Ensure all data is numeric
data = data.apply(pd.to_numeric)

In [34]:
# VAR Model
model_var = VAR(data)
results_var = model_var.fit(maxlags=15, ic='aic')

In [35]:
# Print summary of VAR model results
print(results_var.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 25, Jul, 2024
Time:                     16:56:42
--------------------------------------------------------------------
No. of Equations:         8.00000    BIC:                    2.43241
Nobs:                     763.000    HQIC:                 -0.228828
Log likelihood:          -7226.30    FPE:                   0.151624
AIC:                     -1.89489    Det(Omega_mle):       0.0627259
--------------------------------------------------------------------
Results for equation CRUDE_PETRO
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                  -0.325706         0.357186           -0.912           0.362
L1.CRUDE_PETRO          1.548131         0.319333            4.848           0.000
L1.CRUDE_BRENT         -0.350762         0.245

In [36]:
# Forecast with VAR model
n_forecast = 10  # Number of steps to forecast
forecast_var = results_var.forecast(data.values[-results_var.k_ar:], steps=n_forecast)
forecast_var_df = pd.DataFrame(forecast_var, index=pd.date_range(start=data.index[-1], periods=n_forecast+1, freq='M')[1:], columns=data.columns)
print("VAR Model Forecast:")
print(forecast_var_df)

VAR Model Forecast:
                               CRUDE_PETRO  CRUDE_BRENT  CRUDE_DUBAI  \
1970-02-28 00:00:00.000000773    83.848177    86.538113    84.808626   
1970-03-31 00:00:00.000000773    85.954586    88.411427    87.541744   
1970-04-30 00:00:00.000000773    90.541332    92.775001    92.157464   
1970-05-31 00:00:00.000000773    90.766271    92.547416    93.146474   
1970-06-30 00:00:00.000000773    89.035268    90.706686    90.990440   
1970-07-31 00:00:00.000000773    89.781244    92.137634    90.863046   
1970-08-31 00:00:00.000000773    88.748570    91.345674    90.117612   
1970-09-30 00:00:00.000000773    90.759374    93.146222    92.388558   
1970-10-31 00:00:00.000000773    89.887492    91.870642    91.767537   
1970-11-30 00:00:00.000000773    88.756435    90.848455    90.750287   

                               SUGAR_EU  SUGAR_US         GOLD     PLATINUM  \
1970-02-28 00:00:00.000000773  0.324035  0.866785  2321.045673  1024.758885   
1970-03-31 00:00:00.000000773

In [37]:
# VECM Model
# Perform Johansen cointegration test to determine the number of cointegrating relationships
johansen_test = coint_johansen(data, det_order=0, k_ar_diff=1)
print(johansen_test.lr1)  # Trace statistic
print(johansen_test.cvt)  # Critical values

[375.29607992 262.85033985 164.5078733   96.28450655  52.41371336
  19.97255682   6.29945245   1.12375856]
[[153.6341 159.529  171.0905]
 [120.3673 125.6185 135.9825]
 [ 91.109   95.7542 104.9637]
 [ 65.8202  69.8189  77.8202]
 [ 44.4929  47.8545  54.6815]
 [ 27.0669  29.7961  35.4628]
 [ 13.4294  15.4943  19.9349]
 [  2.7055   3.8415   6.6349]]


In [38]:
# Assuming one cointegrating relationship for VECM
model_vecm = VECM(data, k_ar_diff=1, coint_rank=1)
results_vecm = model_vecm.fit()

In [39]:
# Print summary of VECM model results
print(results_vecm.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation CRUDE_PETRO
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
L1.CRUDE_PETRO     0.3987      0.302      1.321      0.186      -0.193       0.990
L1.CRUDE_BRENT    -0.1107      0.230     -0.482      0.630      -0.561       0.339
L1.CRUDE_DUBAI    -0.0034      0.195     -0.017      0.986      -0.385       0.378
L1.SUGAR_EU        7.8175      5.779      1.353      0.176      -3.510      19.144
L1.SUGAR_US       -1.3273      3.799     -0.349      0.727      -8.773       6.118
L1.GOLD           -0.0078      0.006     -1.330      0.184      -0.019       0.004
L1.PLATINUM        0.0208      0.004      5.811      0.000       0.014       0.028
L1.SILVER         -0.3321      0.147     -2.259      0.024      -0.620      -0.044
Det. terms outside the coint. relation & lagged endog. parameters for equation 

In [40]:
# Forecast with VECM model
forecast_vecm = results_vecm.predict(steps=n_forecast)
forecast_vecm_df = pd.DataFrame(forecast_vecm, index=pd.date_range(start=data.index[-1], periods=n_forecast+1, freq='M')[1:], columns=data.columns)
print("VECM Model Forecast:")
print(forecast_vecm_df)

VECM Model Forecast:
                               CRUDE_PETRO  CRUDE_BRENT  CRUDE_DUBAI  \
1970-02-28 00:00:00.000000773    80.738108    82.362004    81.727244   
1970-03-31 00:00:00.000000773    80.774397    82.682447    81.746082   
1970-04-30 00:00:00.000000773    80.900057    83.016420    81.875180   
1970-05-31 00:00:00.000000773    81.005210    83.272939    81.984696   
1970-06-30 00:00:00.000000773    81.077067    83.454833    82.060060   
1970-07-31 00:00:00.000000773    81.123957    83.581855    82.109538   
1970-08-31 00:00:00.000000773    81.155131    83.671517    82.142566   
1970-09-30 00:00:00.000000773    81.176462    83.735563    82.165236   
1970-10-31 00:00:00.000000773    81.191436    83.781741    82.181181   
1970-11-30 00:00:00.000000773    81.202128    83.815226    82.192578   

                               SUGAR_EU  SUGAR_US         GOLD    PLATINUM  \
1970-02-28 00:00:00.000000773  0.344483  0.838504  2318.723508  984.719716   
1970-03-31 00:00:00.000000773 